## Configuração de Env e Sparks

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/local/openjdk-8"
os.environ["SPARK_HOME"] = "/user_data/spark-3.3.0-bin-hadoop2"

import findspark
findspark.init('spark-3.3.0-bin-hadoop2')

## Imports

In [2]:
# Importando bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import when
from pyspark.sql.functions import unix_timestamp
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

spark = (
    SparkSession.builder.appName("spark_taxi")
    .config("spark.sql.warehouse.dir", "hdfs:///user/hive/warehouse")
    .config("spark.sql.catalogImplementation", "hive")
    .getOrCreate()
)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/user_data/spark-3.3.0-bin-hadoop2/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hadoop-2.7.3/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/05/04 22:31:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Dataset escolhido

O dataset escolhido foi o [NYC Yellow Taxi Trip Data
 (link do Kaggle)](https://www.kaggle.com/datasets/elemento/nyc-yellow-taxi-trip-data?select=yellow_tripdata_2016-03.csv). 

A New York City (NYC) Taxi & Limousine Commission (TLC) disponibiliza dados de todos os seus táxis gratuitamente no website oficial. Esse dataset considera apenas os famosos táxis amarelos de Nova York que oferecem transporte exclusivamente através de chamadas de rua. O csv selecionado possui dados de Janeiro até Março de 2016, sendo 1.91 GB.
 
Dentro do dataset, teremos algumas informações sobre as viagens feitas de táxis amarelos.

## Leitura do dataset

In [3]:
dataframe = spark.read.csv("hdfs://spark-master:9000/datasets/taxi/yellow_tripdata_2016-03.csv", header=True, inferSchema=True)
num_linhas = dataframe.count()
print(f"Número de linhas no DataFrame: {num_linhas}")

# 70% do dataset para rodar no computador
dataframe_sample = dataframe.sample(fraction=0.7, seed=3)
print(f"Número de linhas no DataFrame Sample: {dataframe_sample.count()}")

Número de linhas no DataFrame: 12210952


Número de linhas no DataFrame Sample: 8546588


## Análise Exploratória de Dados (EDA)

In [4]:
# Visualizando o esquema dos dados
dataframe_sample.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



Para melhor entendimento do dataset:

- **VendorID:** Código que indica o TPEP (Taxicab Technology Service Provider) que forneceu o registro. (1= Creative Mobile Technologies, LLC; 2= VeriFone Inc).
- **tpep_pickup_datetime:** Data e hora em que o medidor foi ativado.
- **tpep_dropoff_datetime:** Data e hora em que o medidor foi desativado.
- **passenger_count:** O número de passageiros no veículo. Este é um valor inserido pelo condutor.
- **trip_distance:** A distância percorrida na viagem em milhas, informada pelo taxímetro.
- **pickup_longitude:** Longitude inicial.
- **pickup_latitude:** Latitude inicial.
- **RatecodeID:** O código da taxa final em vigor ao final da viagem. 1 = Taxa padrão, 2 = JFK, 3 = Newark, 4 = Nassau ou Westchester, 5 = Valor negociado, 6 = Corrida de grupo.
- **store_and_fwd_flag:** Esse sinalizador indica se o registro da corrida foi mantido na memória do veículo antes de ser enviado ao fornecedor, também conhecido como "armazenar e encaminhar", porque o veículo não tinha uma conexão com o servidor. Y = armazenar e encaminhar corrida, N = não armazenar e encaminhar corrida.
- **dropoff_longitude:** Longitude final.
- **dropoff_latitude:** Latitude final.
- **payment_type:** Um código numérico que indica como o passageiro pagou pela viagem. 1= Cartão de crédito, 2= Em dinheiro, 3= Sem cobrança, 4= Disputa, 5= Desconhecido, 6= Viagem anulada.
- **fare_amount:** A taxa de tempo e distância calculada pelo medidor.
- **extra:** Extras e taxas adicionais diversas. No momento, inclui somente as cobranças de US\\$ 0,50 e US\$ 1,00 na hora do rush e pernoite.
- **mta_tax:** Taxa de MTA (Metropolitan Transportation Authority) de US\$ 0,50 automaticamente acionada com base na taxa medida em uso.
- **tip_amount:** Este campo é preenchido automaticamente com as gorjetas de cartão de crédito. Gorjetas em dinheiro não são incluídas.
- **tolls_amount:** Valor total de todos os pedágios pagos na viagem.
- **improvement_surcharge:** Taxa adicional de melhoria de US$ 0,30 avaliada em viagens na troca de bandeira. A taxa adicional de melhoria começou a ser cobrada em 2015.
- **total_amount:** O valor total cobrado aos passageiros. Não inclui gorjetas em dinheiro.


In [5]:
# Exibindo algumas informações dos atributos
dataframe_sample.summary().show(truncate=False, vertical=True)

-RECORD 0-------------------------------------
 summary               | count                
 VendorID              | 8546588              
 passenger_count       | 8546588              
 trip_distance         | 8546588              
 pickup_longitude      | 8546588              
 pickup_latitude       | 8546588              
 RatecodeID            | 8546588              
 store_and_fwd_flag    | 8546588              
 dropoff_longitude     | 8546588              
 dropoff_latitude      | 8546588              
 payment_type          | 8546588              
 fare_amount           | 8546588              
 extra                 | 8546588              
 mta_tax               | 8546588              
 tip_amount            | 8546588              
 tolls_amount          | 8546588              
 improvement_surcharge | 8546588              
 total_amount          | 8546588              
-RECORD 1-------------------------------------
 summary               | mean                 
 VendorID    

In [6]:
# Mostrando as primeiras linhas do DataFrame
dataframe.show(n=5, truncate=False, vertical=True)

-RECORD 0------------------------------------
 VendorID              | 1                   
 tpep_pickup_datetime  | 2016-03-01 00:00:00 
 tpep_dropoff_datetime | 2016-03-01 00:07:55 
 passenger_count       | 1                   
 trip_distance         | 2.5                 
 pickup_longitude      | -73.97674560546875  
 pickup_latitude       | 40.76515197753906   
 RatecodeID            | 1                   
 store_and_fwd_flag    | N                   
 dropoff_longitude     | -74.00426483154297  
 dropoff_latitude      | 40.74612808227539   
 payment_type          | 1                   
 fare_amount           | 9.0                 
 extra                 | 0.5                 
 mta_tax               | 0.5                 
 tip_amount            | 2.05                
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 12.35               
-RECORD 1------------------------------------
 VendorID              | 1        

In [7]:
# Checagem por dados nulos
Dict_Null = {col:dataframe_sample.filter(dataframe_sample[col].isNull()).count() for col in dataframe_sample.columns}
Dict_Null

{'VendorID': 0,
 'tpep_pickup_datetime': 0,
 'tpep_dropoff_datetime': 0,
 'passenger_count': 0,
 'trip_distance': 0,
 'pickup_longitude': 0,
 'pickup_latitude': 0,
 'RatecodeID': 0,
 'store_and_fwd_flag': 0,
 'dropoff_longitude': 0,
 'dropoff_latitude': 0,
 'payment_type': 0,
 'fare_amount': 0,
 'extra': 0,
 'mta_tax': 0,
 'tip_amount': 0,
 'tolls_amount': 0,
 'improvement_surcharge': 0,
 'total_amount': 0}

**Pudemos visualizar que não existem dados nulos no dataset.**

## Pré-processamento

In [8]:
# Criar coluna de duração da viagem em horas e remover colunas de datetime
dataframe_sample = dataframe_sample.withColumn(
    "duration_hours",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 3600
)

# Transformar coluna para númerico
dataframe_sample = dataframe_sample.withColumn("store_and_fwd_flag", when(col("store_and_fwd_flag") == 'Y', 1).otherwise(0))

colunas_para_remover = [
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime',
    'VendorID',
    'RatecodeID',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude', 
    'dropoff_latitude',
]

dataframe_pre_processamento = dataframe_sample.drop(*colunas_para_remover)

# Mostrar somente o nome das colunas
print("Nome das colunas:")
for coluna in dataframe_pre_processamento.columns:
    print(coluna)


Nome das colunas:
passenger_count
trip_distance
store_and_fwd_flag
payment_type
fare_amount
extra
mta_tax
tip_amount
tolls_amount
improvement_surcharge
total_amount
duration_hours


**Transformamos as colunas:**
- Datetime de inicio e fim da viagem se transformou em uma coluna de duração da viagem
- Coluna "store_and_fwd_flag" foi convertida para inteiro (0 ou 1)

**Removemos as colunas:**
- tpep_pickup_datetime
- tpep_dropoff_datetime
- VendorID
- RatecodeID
- pickup_longitude
- pickup_latitude
- dropoff_longitude
- dropoff_latitude

Todas essas colunas removidas não possuem muita relação com o valor da gorjeta, são mais relacionadas a localização e classes mais especificas de regras de taxa dos serviços de taxi de Nova Iorque, por isso foram removidas.

In [9]:
# Correlação
assembler = VectorAssembler(inputCols=dataframe_pre_processamento.columns, outputCol="features")
dataframe_vetorizado = assembler.transform(dataframe_pre_processamento)

correlation = Correlation.corr(dataframe_vetorizado, "features", method="pearson").collect()[0][0]

rows = correlation.toArray().tolist()
spark.createDataFrame(rows,dataframe_pre_processamento.columns).show(n=20, truncate=False, vertical=True)

24/05/04 22:35:12 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/04 22:35:12 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


-RECORD 0---------------------------------------
 passenger_count       | 1.0                    
 trip_distance         | -1.9434082309276876E-4 
 store_and_fwd_flag    | -0.022351120635306858  
 payment_type          | 0.01325748102735124    
 fare_amount           | 7.270773640208358E-4   
 extra                 | 0.0020851410589041446  
 mta_tax               | 0.0041108918603492434  
 tip_amount            | -0.004537185300146029  
 tolls_amount          | 0.009392007497496913   
 improvement_surcharge | -0.0027860965125029516 
 total_amount          | 7.615937495781594E-4   
 duration_hours        | 0.006870709087024202   
-RECORD 1---------------------------------------
 passenger_count       | -1.9434082309276876E-4 
 trip_distance         | 1.0                    
 store_and_fwd_flag    | 3.3612935466174053E-6  
 payment_type          | -9.10713706379388E-5   
 fare_amount           | 1.5084119351689823E-4  
 extra                 | 5.779455703506908E-5   
 mta_tax            

In [10]:
# Criando um vetor de features para os modelos
target = 'tip_amount'
feature_columns = dataframe_pre_processamento.columns
feature_columns.remove(target)

vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
dataframe_vetorizado_sem_target = vector_assembler.transform(dataframe_pre_processamento)

In [11]:
# Dividindo o conjunto de dados em treino e teste
train_data, test_data = dataframe_vetorizado_sem_target.randomSplit([0.8, 0.2], seed=3)

## Regressão linear

In [12]:
regressor = LinearRegression(featuresCol = 'features', labelCol = 'tip_amount', loss='squaredError', maxIter=500, regParam = 0.05)
regressor = regressor.fit(train_data)

24/05/04 22:35:42 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [13]:
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show(n=5, truncate=False, vertical=True)

# Printing the results
print('MAE:', pred_results.meanAbsoluteError)
print('MSE:', pred_results.meanSquaredError)
print('RMSE:', pred_results.rootMeanSquaredError)
print('R2-Score:', pred_results.r2)

-RECORD 0------------------------------------------------------------------------------
 passenger_count       | 0                                                             
 trip_distance         | 0.0                                                           
 store_and_fwd_flag    | 0                                                             
 payment_type          | 1                                                             
 fare_amount           | 18.5                                                          
 extra                 | 0.0                                                           
 mta_tax               | 0.5                                                           
 tip_amount            | 2.0                                                           
 tolls_amount          | 0.0                                                           
 improvement_surcharge | 0.3                                                           
 total_amount          | 21.3   

### Árvore de decisão

In [14]:
dt = DecisionTreeRegressor(featuresCol = 'features', labelCol = 'tip_amount', maxDepth=3)
model = dt.fit(train_data)

24/05/04 22:36:56 WARN MemoryStore: Not enough space to cache rdd_253_15 in memory! (computed 7.8 MiB so far)
24/05/04 22:36:56 WARN MemoryStore: Not enough space to cache rdd_253_0 in memory! (computed 3.5 MiB so far)
24/05/04 22:36:56 WARN MemoryStore: Not enough space to cache rdd_253_2 in memory! (computed 3.5 MiB so far)
24/05/04 22:36:56 WARN MemoryStore: Not enough space to cache rdd_253_6 in memory! (computed 5.2 MiB so far)
24/05/04 22:36:56 WARN BlockManager: Persisting block rdd_253_0 to disk instead.
24/05/04 22:36:56 WARN BlockManager: Persisting block rdd_253_6 to disk instead.
24/05/04 22:36:56 WARN BlockManager: Persisting block rdd_253_2 to disk instead.
24/05/04 22:36:56 WARN BlockManager: Persisting block rdd_253_15 to disk instead.
24/05/04 22:36:56 WARN MemoryStore: Not enough space to cache rdd_253_11 in memory! (computed 1024.6 KiB so far)
24/05/04 22:36:56 WARN BlockManager: Persisting block rdd_253_11 to disk instead.
24/05/04 22:36:56 WARN MemoryStore: Not eno

24/05/04 22:37:08 WARN MemoryStore: Not enough space to cache rdd_253_15 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:08 WARN MemoryStore: Not enough space to cache rdd_253_7 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:08 WARN MemoryStore: Not enough space to cache rdd_253_6 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:08 WARN MemoryStore: Not enough space to cache rdd_253_10 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:08 WARN MemoryStore: Not enough space to cache rdd_253_4 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:08 WARN MemoryStore: Not enough space to cache rdd_253_11 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:08 WARN MemoryStore: Not enough space to cache rdd_253_12 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:08 WARN MemoryStore: Not enough space to cache rdd_253_8 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:08 WARN MemoryStore: Not enough space to cache rdd_253_5 in memory! (computed 19.3 MiB so far)
24/05/

24/05/04 22:37:09 WARN MemoryStore: Not enough space to cache rdd_253_0 in memory! (computed 28.9 MiB so far)
24/05/04 22:37:09 WARN MemoryStore: Not enough space to cache rdd_253_1 in memory! (computed 28.9 MiB so far)
24/05/04 22:37:09 WARN MemoryStore: Not enough space to cache rdd_253_3 in memory! (computed 28.9 MiB so far)
24/05/04 22:37:09 WARN MemoryStore: Not enough space to cache rdd_253_14 in memory! (computed 28.9 MiB so far)
24/05/04 22:37:09 WARN MemoryStore: Not enough space to cache rdd_253_2 in memory! (computed 28.9 MiB so far)


24/05/04 22:37:10 WARN MemoryStore: Not enough space to cache rdd_253_4 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:10 WARN MemoryStore: Not enough space to cache rdd_253_14 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:10 WARN MemoryStore: Not enough space to cache rdd_253_13 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:10 WARN MemoryStore: Not enough space to cache rdd_253_7 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:10 WARN MemoryStore: Not enough space to cache rdd_253_9 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:10 WARN MemoryStore: Not enough space to cache rdd_253_8 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:10 WARN MemoryStore: Not enough space to cache rdd_253_5 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:10 WARN MemoryStore: Not enough space to cache rdd_253_15 in memory! (computed 19.3 MiB so far)
24/05/04 22:37:10 WARN MemoryStore: Not enough space to cache rdd_253_0 in memory! (computed 19.3 MiB so far)
24/05/0

In [15]:
pred_results = model.transform(test_data)
pred_results.show(n=5, truncate=False, vertical=True)

# Printing the results
evaluator = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'tip_amount')

mae = evaluator.evaluate(pred_results,
{evaluator.metricName: "mae"})

mse = evaluator.evaluate(pred_results,
{evaluator.metricName: "mse"})

rmse = evaluator.evaluate(pred_results,
{evaluator.metricName: "rmse"})

r2 = evaluator.evaluate(pred_results,
{evaluator.metricName: "r2"})

print('MAE:', mae)
print('MSE:', mse)
print('RMSE:', rmse)
print('R2-Score:', r2)

-RECORD 0------------------------------------------------------------------------------
 passenger_count       | 0                                                             
 trip_distance         | 0.0                                                           
 store_and_fwd_flag    | 0                                                             
 payment_type          | 1                                                             
 fare_amount           | 18.5                                                          
 extra                 | 0.0                                                           
 mta_tax               | 0.5                                                           
 tip_amount            | 2.0                                                           
 tolls_amount          | 0.0                                                           
 improvement_surcharge | 0.3                                                           
 total_amount          | 21.3   

MAE: 0.5628859527250294
MSE: 1.8051962967944253
RMSE: 1.3435759363707083
R2-Score: 0.7141497763903443
